<a href="https://colab.research.google.com/github/siwookim1114/Deep-Learning-with-Keras-and-TensorFlow/blob/main/%08Ch3_%EC%BC%80%EB%9D%BC%EC%8A%A4%EC%99%80_%ED%85%90%EC%84%9C%ED%94%8C%EB%A1%9C_%EC%86%8C%EA%B0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 텐서플로 시작하기

## GradientTape API
* 미분 가능한 표현이라면 어떤 입력에 대해서도 그레이디언트를 계산할 수 있음
* gradient = tape.gradient(loss, weights) 와 같이 가중치에 대한 모델 손실의 그레이디언트를 계산하는데 가장 널리 사용되는 방법


In [ ]:
import tensorflow as tf
input_var = tf.Variable(initial_value = 3,)
with tf.GradientTape() as tape:
  result = tf.square(input_var)
gradient = tape.gradient(result, input_var)

# 신경망의 구조 이해하기

## 층 : 딥러닝의 구성 요소
* 신경망의 기본 데이터 구조 => 층 (layer)
    * 하나 이상의 텐서를 입력받고 하나 이상의 텐서를 출력하는 데이터 처리 모듈
* 텐서의 크기 별 층 처리:
    * Rank-2 텐서 (samples, features) => Dense Layer
    * Rank -3 텐서 (samples, timesteps, features) => Recurrent Layer (LSTM 같은거) / 1D Convolution Layer
    * Rank -4 텐서 (이미지 데이터 등) => 2D Convolution Layer


### Dense Layer 코드로 구현

In [ ]:
from tensorflow import keras

class SimpleDense(keras.layers.Layer):    ## 모든 케라스 층은 Layer 클래스를 상속함
  def __init__(self, units, activation = None):
    super().__init__()
    self.units = units
    self.activation = activation

  def build(self, input_shape):   ## build() 메서드에서는 가중치를 생성함
    input_dim = input_shape[-1]
    self.W = self.add_weight(shape = (input_dim, self.units), ## add_weight()는 가중치를 간편하게 만드는 메서드.
                             initializer = "random_normal")
    self.b = self.add_weight(shape = (self.units,),
                             initializer = "zeros")

  def call(self, inputs): ## call() 메서드에서는 정방향 패스 계산을 정의함
    y = tf.matmul(inputs, self.W) + self.b
    if self.activation is not None:
      y = self.activation(y)
    return y

In [ ]:
my_dense = SimpleDense(units = 32, activation = tf.nn.relu) ## 층의 인스턴스 생성
input_tensor = tf.ones(shape = (2, 784))  ## 텍스트 입력
output_tensor = my_dense(input_tensor) #층을 함수처럼 호출

print(output_tensor.shape)

(2, 32)


## 컴파일 단계 : 학습 과정 설정
* 손실함수 (Loss function) : 훈련 과정에서 최소화 할 값 => 현재 작업에 대한 성공의 척도
    * CategoricalCrossentropy
    * SparseCategoricalCrossentropy
    * BinaryCrossentropy
    * MeanSquaredError
    * KLDivergence
    * CosineSimilarity
    * 그 외
* 옵티마이저 (Optimizer) : 손실 함수를 기반으로 네트워크가 어떻게 업데이트될지 결정함 => 특정 종류의 경사 하강법 (SGD)로 구현
    * SGD (모멘텀 선택 가능)
    * RMSprop
    * Adam
    * Adagrad
    * 그 외
    
* 측정지표 (Metric) : 훈련과 검증 과정에서 모니터링할 성공의 척도 => e.g) 분류 정확도
    * CategoricalAccuracy
    * SparseCategoricalAccuracy
    * BinaryAccuracy
    * AUC
    * Precision
    * Recall
    * 그 외

In [ ]:
# compile() => 훈련 과정을 설정.
## 매개변수는 optimizer, loss, metrics(리스트)
model = keras.Sequential([keras.layers.Dense(1)])   ## 선형 분류기 정의
model.compile(optimizer = "rmsprop",                ## 옵티마이저 이름 설정
              loss = "mean_squared_error",          ## 손실 이름 : 평균 제곱 오차로 지정
              metrics = ["accuracy"])               ## 측정 자료를 리스트로 지정. 여기에서는 정확도만 사용


In [ ]:
# 매개변수들이 문자열로 지정되는게 아니라 실제로는 함수이다
## e.g) "rmsprop" => keras.optimizers.RMSprop()
## 함수로 지정하는 경우에는 사용자 정의 손실이나 측정 지표를 전달하고 싶을때 유용함. 또는 사용할 객체를 상세히 설정하고 싶을때. (learning_rate 매개변수 바꾸기 등)
model.compile(optimizer = keras.optimizers.RMSprop(),
              loss = keras.losses.MeanSquaredError(),
              metrics = [keras.metrics.BinaryAccuracy()])


model.compile(optimizer = keras.optimizers.RMSprop(learning_rate = 1e-4),
              loss = my_custom_loss,
              metrics = [my_custom_metric_1, my_custom_metric_2])

## 손실 함수 선택
* 보통 2개의 클래스가 있는 분류 문제 => BinaryCrossEntropy
* 여러개의 클래스가 있는 분류 문제 => Categorical Cross Entropy


## fit() 메서드 => 훈련 루프 구현
* 매개변수:
    * 훈련할 데이터 (입력과 타깃) : 보통 넘파이 배열이나 텐서플로 Dataset 객체로 전달
    * 훈련할 epoch 횟수 : 전달한 데이터에서 훈련 루프를 몇 번이나 반복할지
    * 배치 : 가중치 업데이트 단게에서 그레이디언트를 계산하는 데 사용될 훈련 샘플 개수


In [ ]:
history = model.fit(
    inputs,    ## 입력 샘플
    targets,   ## 훈련 타깃 (넘파이 배열)
    epochs = 5, ## 훈련 루프 다섯번 반복
    batch_size = 128    ## 훈련 루프는 128개의 샘플 배치로 데이터 순회
)

In [ ]:
# fit() 을 호출하면 History 객체가 반환됨.
## 이건 history (dict) 속성 가지고 있음 => "loss" 또는 특정 측정 지표 이름의 키와 각 에포크 값의 리스트를 매핑
history.history

## 검증 데이터에서 손실과 측정 지표 모니터링
* 새로운 데이터에 모델이 어떻게 동작하는지 예상하기 위해 훈련 데이터의 일부를 validation data로 떼어 놓음
    * 검증 데이터에서 모델 훈련은 X => 이 데이터를 사용하여 손실과 측정 지표를 계산
    * fit() 에서 validation_data 매개변수 사용

In [ ]:
model.fit(
    training_inputs,
    training_targets,
    epochs = 5,
    batch_size = 16,
    validation_data = (validation_inputs, validation_targets)  ## 검증 데이터는 검증 손실과 측정 지표를 모니터링 하는데 사용
)

* 훈련이 끝난 후:
    * 검증 데이터의 손실 값을 Training Loss 과 구분하기 위해 Validation Loss라고 부름
    * 훈련 데이터와 검증 데이터를 엄격하게 분리하는 것은 필수
        * 검증 목적은 모델이 학습한 것이 새로운 데이터에 실제로 유용한지 모니터링하는 것이기 때문

* 훈련이 끝난 후 validation loss과 측정 지표를 계산하고 싶다면 evaluate() 메서드 사용


In [ ]:
# evaluate() 메서드는 전달된 데이터를 배치 사이즈의 크기로 순쇠하고 스칼라 값의 리스트를 반환함
## 반환된 리스트의 첫 번째 항목은 validation loss, 그 다음이 검증 데이터에 대한 측정 지표 값
loss_and_metrics = model.evaluate(validation_inputs, validation_targets, batch_size = 128)